In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
from scipy.stats import shapiro

# Zadanie 1 (4 pkt)
Przeprowadź test T studenta na danych w pliku `wyniki.csv`. \
<u>Hipoteza zerowa</u> brzmi: Nie ma istotnej różnicy w ocenie końcowej pomiędzy uczniami i uczennicami. \
<u>Hipoteza alternatywna</u> brzmi: Istnieje istotna różnica (bez wskazania kierunku) w ocenie końcowej pomiędzy uczniami i uczennicami. \
Przyjmij significance level $\alpha=0.05$. Czy uda się odrzucić hipotezę zerową?

In [ ]:

ALPHA = 0.05

df = pd.read_csv("data/08/wyniki.csv")

m = df[df["plec"] == "M"]["ocena_koncowa"]
f = df[df["plec"] == "F"]["ocena_koncowa"]

# sprawdzamy czy dane są rozkładu normalnego
# pierwszy sposób - test shapiro-wilka (czy pvalue > alpha)
shapiro_m = shapiro(m)
m_ok = shapiro_m.pvalue > ALPHA
shapiro_f = shapiro(f)
f_ok = shapiro_m.pvalue > ALPHA
print(f"Shapiro-Wilk pvalue (M): {shapiro_m.pvalue}, OK: {m_ok}")
print(f"Shapiro-Wilk pvalue (F): {shapiro_f.pvalue}, OK: {f_ok}")
# drugi sposób - z histogramu na oko (widać że niezbyt)
fig_m = go.Figure()
fig_m.add_trace(go.Histogram(x=m, name="M"))
fig_m.show()

fig_f = go.Figure()
fig_f.add_trace(go.Histogram(x=f, name="F"))
fig_f.show()

# założenia testu t-studenta nie są spełnione, więc można odrzucić hipotezę zerową

# Zadanie 2 (2 pkt)
Korzystając z danych w pliku `ZyskiFirmyX.csv`, stwórz model regresji liniowej i biblioteki `statsmodels`:
- Wyświetl wykres pokazujący zyski w czasie
- Czy da się zaobserwować trend? Jaki?
- Wyświetl podsumowanie modelu

In [ ]:
df = pd.read_csv("data/08/ZyskiFirmyX.csv")

X = sm.add_constant(df["Rok"])
Y = df["Zysk"]
model = sm.OLS(Y, X).fit()
df["Trend"] = model.predict(X)

fig = px.scatter(df, x="Rok", y="Zysk")
fig.add_trace(go.Scatter(x=df["Rok"], y=df["Trend"], mode="lines", name="Trend"))
fig.update_layout(xaxis_title="Rok", yaxis_title="Zysk")
fig.show()
# trend jest bardzo delikatnie spadkowy

model.summary()

# Zadanie 3 (6 pkt)
Korzystając z danych w pliku `spendingscores.csv` i biblioteki `statsmodels`:
1. Stwórz model regresji liniowej gdzie zmienną zależną jest `Spending score`, a zmiennymi niezależnymi pozostałe zmienne
2. Z modelu wyświetl:
    - p values
    - standard dev
    - coefficients
3. Sprawdź czy istnieje korelacja pomiędzy zmiennymi. Pokaż to na wykresie
4. Korzystając z eliminacji wstecznej, usuń najmniej istotną zmienną i jeszcze raz wyświetl model

In [ ]:
def print_model_summary(model):
    print(f"P values:\n{model.pvalues}")
    print(f"Standard dev:\n{model.bse}")
    print(f"Coefficients:\n{model.params}")


df = pd.read_csv("data/08/spendingscores.csv")

df["Gender"] = df["Gender"].map({"Male": 1, "Female": 0})

x = sm.add_constant(df[["Gender", "Age", "AnnualIncome"]])
y = df["SpendingScore"]
model = sm.OLS(y, x).fit()
print_model_summary(model)

corr = df.corr()
fig = px.imshow(corr, text_auto=True, title="Correlation matrix")
fig.show()

p_values = model.pvalues
max_p_value = p_values.max()
excluded_var = p_values.idxmax()
print(f"Najmniej istotna zmienna: {excluded_var} (p-value: {max_p_value})")
x = x.drop(columns=[excluded_var])
model = sm.OLS(y, x).fit()
print_model_summary(model)